In [1]:
import sys
sys.path.append("../code")
from dataset import build_dataset, build_dataloader
from config import DataArguments, TrainerArguments, ModelArguments

In [13]:
data_args = DataArguments()
training_args = TrainerArguments()
model_args = ModelArguments()

data_args.data_dir = "../data/"
data_args.asset_dir = "../assets/"
model_args.asset_dir = data_args.asset_dir
data_args.seed = training_args.seed
data_args.max_seq_len = model_args.max_seq_len

data_args.init_pct = 1
training_args.do_train = True
training_args.use_gpu = False

train_dataset, model_args.vocab_size, model_args.num_labels = build_dataset(data_args, "train")
train_dataloader = build_dataloader(train_dataset, data_args)

[11/24/2021 13:45:48] INFO - dataset: Initialize Train Dataset.
[11/24/2021 13:45:48] INFO - dataset: Remove abstract.
[11/24/2021 13:45:48] INFO - dataset: Using cached dataset, wasn't able to remove columns.
[11/24/2021 13:45:48] INFO - dataset: Use the full dataset, for train dataset of total 47250 papers.
[11/24/2021 13:45:48] INFO - dataset: Train dataset was successfully initialized.
[11/24/2021 13:45:48] INFO - dataset: Successfully loaded mapper file ..\assets\area2idx.json
[11/24/2021 13:45:48] INFO - dataset: Use tokenized_paperswithcode dataset.
[11/24/2021 13:45:48] INFO - dataset: Preprocessed dataset. Use default vocab_size=83931.
[11/24/2021 13:45:48] INFO - dataset: Successfully converted dataset to dataloader.


In [14]:
from trainer import NaiveTrainer
trainer= NaiveTrainer(
    data_args,
    training_args,
    model_args,
    training_dataset=train_dataloader if training_args.do_train else None,
    validation_dataset=valid_dataloader if training_args.do_valid else None,
    test_dataset=test_dataloader if training_args.do_test else None,
)

[11/24/2021 13:45:49] INFO - trainer: Bert was selected. Start Transformer setup.
[11/24/2021 13:45:49] INFO - trainer: Successfully setup transformer settings.


In [15]:
import torch
import torch.nn as nn

loss_fc = nn.BCEWithLogitsLoss()

In [17]:
batch = next(iter(train_dataloader))

In [19]:
logit, prediction = trainer.model(batch)

In [43]:
torch.nn.functional.one_hot(batch["labels"])

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [28]:
loss = loss_fc(logit, torch.nn.functional.one_hot(batch["labels"]).type(torch.float))

In [42]:
batch["labels"]

tensor([ 0,  7, 13, 10,  7,  7,  7, 10,  2, 15,  7, 14,  7, 15, 10, 10,  7,  4,
        14, 14,  8,  7,  3, 14, 12, 10,  8, 12, 10,  7, 15,  7,  9,  7, 14,  3,
        12, 15,  4, 15, 14,  7, 10,  7,  7,  7,  4,  5,  4, 12,  3,  7, 15,  3,
        14, 12, 12, 10,  7, 12,  3,  3, 15,  7,  8, 15,  3,  7,  2, 14,  3,  7,
         3, 10, 12,  3,  7, 15, 14, 15, 13,  7,  4, 12,  7,  7,  4, 14,  3, 10,
         8,  0, 15,  7,  3,  7,  8,  7, 15,  7, 15, 14, 15, 15,  4,  3, 14, 12,
        15,  3,  1,  5,  3,  3, 10,  4, 10, 10,  7,  6, 10,  7,  3,  5,  6, 10,
        10, 11])

In [31]:
bce = nn.BCELoss()

In [39]:
logit

tensor([[ 0.0227, -0.1007,  0.0594,  ..., -0.1450,  0.2345,  0.0408],
        [-0.0349, -0.0310,  0.0113,  ...,  0.0155,  0.3134,  0.0341],
        [ 0.1405, -0.1610, -0.0071,  ..., -0.1379,  0.2116,  0.1854],
        ...,
        [ 0.1185, -0.0879,  0.0674,  ..., -0.0012,  0.2268,  0.0956],
        [ 0.0611, -0.0226,  0.1324,  ..., -0.0433,  0.2603,  0.1079],
        [ 0.0659, -0.1044,  0.0750,  ...,  0.0548,  0.2539, -0.0188]],
       grad_fn=<AddmmBackward>)

In [41]:
"""
This module demos the LogSumExp trick. See https://blog.feedly.com/?p=10329
"""
import math
from typing import List
import logging
import time


def log_sum_exp_naive(X:List[float]) -> float:
    """
    a naive calculation of LogSumExp expressions
    :param X: a list of numbers
    :return: the LogSumExp calculation
    """
    logging.debug('START lse_naive(%s)', X)
    try:
        summation = 0
        for x_i in X:
            v = math.e**x_i
            logging.debug('e^%f = %.5f', x_i, v)
            summation += v
        return math.log(summation)
    except Exception as e:
        logging.debug('lse_naive FAILURE')
        raise e


def log_sum_exp(X:List[float]) -> float:
    """
    a better calculation of LogSumExp expressions
    :param X: a list of numbers
    :return: the LogSumExp calculation
    """
    logging.debug('START lse(%s)', X)
    c = max(X)
    summation = 0
    for x_i in X:
        v = math.e ** (x_i - c)
        logging.debug('e^(%f - c) = %.5f', x_i, v)
    summation += sum(math.e ** (x_i - c) for x_i in X)

    logging.debug('c=%.5f; summation=%.5f', c, summation)

    return math.log(summation) + c


def log_softmax(j:int, X:List[float], naive:bool=False) -> float:
    """
    a log softmax calculation
    :param j: an index into X that selects the numerator value.
    :param X: a list of numbers
    :param naive: use the naive LogSumExp method
    :return: the log softmax calculation
    """
    lse = log_sum_exp_naive if naive else log_sum_exp
    return X[j] - lse(X)


if __name__ == '__main__':
    logging.basicConfig(level='INFO')  # change to debug to print intermediate calculations

    def _run_example(j:int, X:List[float]) -> None:
        print('*' * 30)
        print(f'* X={X}')
        print(f'* j={j}\n')
        time.sleep(0.001) # so the logs get printed out nicely
        y1 = log_sum_exp(X)
        try:
            y2 = log_sum_exp_naive(X)
            if abs(y1 - y2) > 1e-6:
                raise ValueError(f'calculation error {y1} != {y2}')
        except:
            y2 = 'bombed!'

        print(f'logsumpexp({X}): {y1}')
        print(f'logsumpexp({X}): {y2} (naive)')

        ls = log_softmax(j, X)
        print(f'log(softmax({j}, {X}) = {ls} --> softmax = {math.e**ls}')
        if isinstance(y2, float):
            ls = log_softmax(j, X, True)
            print(f'log(softmax({j}, {X}, naive) = {ls}')

        print('*' * 30,'\n')

    # the examples from the blog post plus a small numerically stable example
    _examples = [[1000]*3, [-1000]*3, [1,1,1]]

    for _example in _examples:
        _run_example(0, _example)

    # one huge X value
    _run_example(0, [1000, 1, 2, 3])

    # one huge negative X value
    _run_example(0, [-1000, 1, 2, 3])

    # run this in debug mode to see what happens to the contributions of the values < 1 in the logsumexp calculation and
    # also what happens to the softmax probability distribution.
    _run_example(0, [1000, 1e-5, 1e-10])
    _run_example(1, [1000, 1e-5, 1e-10])
    _run_example(2, [1000, 1e-5, 1e-10])

******************************
* X=[1000, 1000, 1000]
* j=0

logsumpexp([1000, 1000, 1000]): 1001.0986122886682
logsumpexp([1000, 1000, 1000]): bombed! (naive)
log(softmax(0, [1000, 1000, 1000]) = -1.0986122886681642 --> softmax = 0.33333333333331516
****************************** 

******************************
* X=[-1000, -1000, -1000]
* j=0

logsumpexp([-1000, -1000, -1000]): -998.9013877113318
logsumpexp([-1000, -1000, -1000]): bombed! (naive)
log(softmax(0, [-1000, -1000, -1000]) = -1.0986122886681642 --> softmax = 0.33333333333331516
****************************** 

******************************
* X=[1, 1, 1]
* j=0

logsumpexp([1, 1, 1]): 2.09861228866811
logsumpexp([1, 1, 1]): 2.0986122886681096 (naive)
log(softmax(0, [1, 1, 1]) = -1.09861228866811 --> softmax = 0.33333333333333326
log(softmax(0, [1, 1, 1], naive) = -1.0986122886681096
****************************** 

******************************
* X=[1000, 1, 2, 3]
* j=0

logsumpexp([1000, 1, 2, 3]): 1000.0
logsumpexp([1000

In [47]:
import math
from dataclasses import dataclass
from typing import List

import torch
from toma import toma
from tqdm.auto import tqdm

from batchbald_redux import joint_entropy

K = 20

In [48]:
import numpy as np


def get_mixture_prob_dist(p1, p2, m):
    return (1.0 - m) * np.asarray(p1) + m * np.asarray(p2)


p1 = [0.7, 0.1, 0.1, 0.1]
p2 = [0.3, 0.3, 0.2, 0.2]
y1_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.7, 0.1, 0.1]
p2 = [0.2, 0.3, 0.3, 0.2]
y2_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.1, 0.7, 0.1]
p2 = [0.2, 0.2, 0.3, 0.3]
y3_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.1, 0.1, 0.7]
p2 = [0.3, 0.2, 0.2, 0.3]
y4_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]


def nested_to_tensor(l):
    return torch.stack(list(map(torch.as_tensor, l)))


ys_ws = nested_to_tensor([y1_ws, y2_ws, y3_ws, y4_ws])

In [49]:
ys_ws

tensor([[[0.7000, 0.1000, 0.1000, 0.1000],
         [0.6789, 0.1105, 0.1053, 0.1053],
         [0.6579, 0.1211, 0.1105, 0.1105],
         [0.6368, 0.1316, 0.1158, 0.1158],
         [0.6158, 0.1421, 0.1211, 0.1211],
         [0.5947, 0.1526, 0.1263, 0.1263],
         [0.5737, 0.1632, 0.1316, 0.1316],
         [0.5526, 0.1737, 0.1368, 0.1368],
         [0.5316, 0.1842, 0.1421, 0.1421],
         [0.5105, 0.1947, 0.1474, 0.1474],
         [0.4895, 0.2053, 0.1526, 0.1526],
         [0.4684, 0.2158, 0.1579, 0.1579],
         [0.4474, 0.2263, 0.1632, 0.1632],
         [0.4263, 0.2368, 0.1684, 0.1684],
         [0.4053, 0.2474, 0.1737, 0.1737],
         [0.3842, 0.2579, 0.1789, 0.1789],
         [0.3632, 0.2684, 0.1842, 0.1842],
         [0.3421, 0.2789, 0.1895, 0.1895],
         [0.3211, 0.2895, 0.1947, 0.1947],
         [0.3000, 0.3000, 0.2000, 0.2000]],

        [[0.1000, 0.7000, 0.1000, 0.1000],
         [0.1053, 0.6789, 0.1105, 0.1053],
         [0.1105, 0.6579, 0.1211, 0.1105],
         

In [90]:
def to_2d(logits):

    if logits.ndim == 3 and logits.shape[1] == 1:
        logits = logits.squeeze()
        return logits
    elif logits.ndim == 3 and logits.shape[1] > 1:
        raise


In [93]:
_logits = to_2d(logits)
_logits

tensor([[0.7000, 0.1000, 0.1000, 0.1000, 0.0000],
        [0.3000, 0.3000, 0.2000, 0.2000, 0.0000]])

In [87]:
def margin_of_confidence(logits: np.ndarray):

    logits = to_2d(logits)
    print(logits)

    part = torch.split(-logits, 1, dim=1)[0].squeeze()

    print(part)
    margin = -part[:, 0] + part[:, 1]
    return margin

In [80]:
def margin_of_confidence_np(logits: np.ndarray):

    part = np.partition(-logits, 1, axis=1)
    print(part)
    margin = -part[:, 0] + part[:, 1]
    return margin

In [81]:
import torch
p1 = [0.7, 0.1, 0.1, 0.1, 0.0]
p2 = [0.3, 0.3, 0.2, 0.2, 0.0]

logits = torch.stack(tuple(map(torch.tensor, (p1, p2)))).unsqueeze(1)

In [89]:
logits.shape

torch.Size([2, 1, 5])

In [82]:
import numpy as np
logits_np = np.array((p1, p2))

In [83]:
margin_of_confidence_np(logits_np)

[[-0.7 -0.1 -0.1 -0.1 -0. ]
 [-0.3 -0.3 -0.2 -0.2 -0. ]]


array([0.6, 0. ])

In [88]:
margin_of_confidence(logits)

tensor([[0.7000, 0.1000, 0.1000, 0.1000, 0.0000],
        [0.3000, 0.3000, 0.2000, 0.2000, 0.0000]])
tensor([-0.7000, -0.3000])


IndexError: too many indices for tensor of dimension 1

In [85]:
part = torch.split(-logits, 1, dim=1)[0].squeeze()

In [86]:
part

tensor([[-0.7000, -0.1000, -0.1000, -0.1000, -0.0000],
        [-0.3000, -0.3000, -0.2000, -0.2000, -0.0000]])

In [73]:
part[:, 1]

tensor([-0.1000, -0.3000])